# Data Aggregation and Group Operations

Thanks to pandas we can *pivot tables* or *group by* to compute group
statistics for reporting or visualization purposes, enabling us to slice, dice, 
and summarize datasets in a natural way.



## Index

- [How to Think About Group Operations](#how-to-think-about-group-operations)
    - [Iterating over groups](#iterating-over-groups)
    - [Selecting a Column or Subset of Columns](#selecting-a-column-or-subset-of-columns)
    - [Grouping with Dictionaries and Series](#grouping-with-dictionaries-and-series)
    - [Grouping by Index Levels](#grouping-by-index-levels)
- [Data Aggregation](#data-aggregation)
    - [Column-Wise and Multiple Function Application](#column-wise-and-multiple-function-application)
    - [Returning Aggregated Data Without Row Indexes](#returning-aggregated-data-without-row-indexes)
- [Apply: General split-apply-combine](#apply-general-split-apply-combine)
    - [Quantile and Bucket Analysis](#quantile-and-bucket-analysis)
    - [Example: Filling Missing Values with Group-Specific Values](#example-filling-missing-values-with-group-specific-values)
    

In [94]:
import numpy as np 
import pandas as pd
import statsmodels.api as sm
#import seaborn as sns
#import matplotlib.pyplot as plt
import warnings
#from datetime import datetime 
from sinfo import sinfo
from io import StringIO

warnings.filterwarnings("ignore")

# matplotlib:
#%matplotlib inline
#plt.rc("figure", figsize=(16,8))

## How to Think About Group Operations

The core is *split-apply-combine*:
1. data in a DataFrame/Series is split into groups based on passed *keys*.
    - Grouped on rows: `(axis="index")`
    - Grouped on columns: `(axis="columns")`
2. A functions is *applied* to each group which results a new value.
3. This results are combined into a new object.

When we use `groupby()` the new variable is a special "GroupBy" object which 
we can compute some operations. 

In [95]:
data = pd.DataFrame({
    "key1" : ["a", "a", None, "b", "b", "a", None],
    "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
    "data1" : np.random.standard_normal(7),
    "data2" : np.random.standard_normal(7),
})
data

,key1,key2,data1,data2
0,a,1,0.206235,0.221009
1,a,2,-0.820793,-0.267532
2,None,1,-0.374071,-0.546836
3,b,2,-1.691274,1.706690
4,b,1,-0.226503,-1.413782
5,a,<NA>,-0.923427,-0.546473
6,None,1,-0.726968,-0.745141


***

`data1` column mean using `key1` labels

***

In [96]:
grpd1k1 = data["data1"].groupby(data["key1"])
grpd1k1

In [97]:
# Mean calculation on grouped variable

grpd1k1.mean()

key1
a   -0.512661
b   -0.958888
Name: data1, dtype: float64

In [98]:
grpd1means = data["data1"].groupby([data["key1"], data["key2"]]).mean()
grpd1means

key1  key2
a     1       0.206235
      2      -0.820793
b     1      -0.226503
      2      -1.691274
Name: data1, dtype: float64

***
From series with hierarchical index to dataframe unstakced
***

In [99]:
grpd1means.unstack()

key2,1,2
key1,,
a,0.206235,-0.820793
b,-0.226503,-1.691274


***
New keys with the same length for our data
***

In [100]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

# If the new series have the same length, we can used as keys for groupby

data["data1"].groupby([states, years]).mean()

CA  2005   -0.872110
    2006   -0.374071
OH  2005   -0.742519
    2006   -0.476735
Name: data1, dtype: float64

***
When the grouping information is in the same DataFrame we can pass the column
names and it will group the rest
***

In [101]:
temp = data.groupby("key1").mean()
temp

,key2,data1,data2
key1,,,
a,1.5,-0.512661,-0.197665
b,1.5,-0.958888,0.146454


In [102]:
temp = data.groupby(["key2", "key1"]).mean()
temp

data1     data2
key2 key1                    
1    a     0.206235  0.221009
     b    -0.226503 -1.413782
2    a    -0.820793 -0.267532
     b    -1.691274  1.706690

***
GroupBy and `size()` method is useful to return group sizes.

`count()` computes the number of nonnull values in each group
***

In [103]:
temp = data.groupby("key1", dropna=False).size()
temp

key1
a      3
b      2
NaN    2
dtype: int64

In [104]:
temp = data.groupby(["key1", "key2"], dropna=False).size()
temp


key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [105]:
temp = data.groupby("key1").count()
temp

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


### Iterating over Groups

In [106]:
data

,key1,key2,data1,data2
0,a,1,0.206235,0.221009
1,a,2,-0.820793,-0.267532
2,None,1,-0.374071,-0.546836
3,b,2,-1.691274,1.706690
4,b,1,-0.226503,-1.413782
5,a,<NA>,-0.923427,-0.546473
6,None,1,-0.726968,-0.745141


In [107]:
for name, group in data.groupby("key1"):
    print(name)
    print(group)


a
  key1  key2     data1     data2
0    a     1  0.206235  0.221009
1    a     2 -0.820793 -0.267532
5    a  <NA> -0.923427 -0.546473
b
  key1  key2     data1     data2
3    b     2 -1.691274  1.706690
4    b     1 -0.226503 -1.413782


In [108]:
for (k1, k2), group in data.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', 1)
  key1  key2     data1     data2
0    a     1  0.206235  0.221009
('a', 2)
  key1  key2     data1     data2
1    a     2 -0.820793 -0.267532
('b', 1)
  key1  key2     data1     data2
4    b     1 -0.226503 -1.413782
('b', 2)
  key1  key2     data1    data2
3    b     2 -1.691274  1.70669


***
Can be useful create a dictionary with the data
***


In [109]:
data_pieces = {name: group for name, group in data.groupby("key1")}
print(data_pieces["b"])
print("\n", data_pieces["a"], sep='')

  key1  key2     data1     data2
3    b     2 -1.691274  1.706690
4    b     1 -0.226503 -1.413782

  key1  key2     data1     data2
0    a     1  0.206235  0.221009
1    a     2 -0.820793 -0.267532
5    a  <NA> -0.923427 -0.546473


### Selecting a column or subset of Columns

Sometimes in large datasets we'll prefer to aggregate a few columns to compute
some calculations.

In [110]:
data.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     0.221009
     2    -0.267532
b    1    -1.413782
     2     1.706690

### Grouping with Dictionaries and Series

We can group passing a dictionary and then operate with that, e.g. mean, sum, 
count...

In [111]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])

people.iloc[2:3, [1, 2]] = np.nan

people

,a,b,c,d,e
Joe,0.313566,-0.606102,-0.309854,0.721010,1.557726
Steve,-0.694519,0.414162,1.788449,0.548465,0.817504
Wanda,-1.373022,NaN,NaN,0.136258,0.081706
Jill,1.020792,1.027509,0.232399,-0.182039,1.062498
Trey,1.540700,-1.278000,-0.234004,0.612878,-0.232986


In [112]:
mapping = {"a":"red", "b": "red", "c": "blue", 
           "d": "blue", "e": "red", "f" : "orange"}

sum_col = people.groupby(mapping, axis="columns")
sum_col.sum()

,blue,red
Joe,0.411157,1.265190
Steve,2.336914,0.537147
Wanda,0.136258,-1.291316
Jill,0.050360,3.110798
Trey,0.378874,0.029714


In [113]:
sum_col.mean()

,blue,red
Joe,0.205578,0.421730
Steve,1.168457,0.179049
Wanda,0.136258,-0.645658
Jill,0.025180,1.036933
Trey,0.189437,0.009905


In [114]:
map_ser = pd.Series(mapping)
map_ser

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [115]:
people.groupby(map_ser, axis="columns").count()

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Grouping by Index Levels

To group by level we can pass the level number or level name as argument:

`df.groupby(level='1', axis='1')`.

In [116]:
cols = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                 [1, 3, 7, 1, 3]], names=["cty", "tenor"])
hier_data = pd.DataFrame(np.random.standard_normal((4, 5)), columns=cols)
hier_data

cty          US                            JP          
tenor         1         3         7         1         3
0     -0.289291  2.991137  0.009767 -1.265774 -0.064203
1     -0.473549 -1.260517 -1.188470  0.154258 -0.349685
2     -1.141150  0.259794 -1.223113  0.771600 -0.558552
3      0.135691 -0.251392 -1.895319 -1.520158 -0.946721

In [117]:
hier_data.groupby(level="cty", axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation

Aggregations refer to any data transformation that produces scalar values from arrays.


*Optimized GroupBy methods*
| Function name| Description|
|--:|---|
|any, all |Return True if any (one or more values) or all non-NA values are “truthy”|
|count |Number of non-NA values|
|cummin, cummax |Cumulative minimum and maximum of non-NA values|
|cumsum |Cumulative sum of non-NA values|
|cumprod |Cumulative product of non-NA values|
|first, last |First and last non-NA values|
|mean |Mean of non-NA values|
|median |Arithmetic median of non-NA values|
|min, max |Minimum and maximum of non-NA values|
|nth |Retrieve value that would appear at position n with the data in sorted order|
|ohlc |Compute four “open-high-low-close” statistics for time series-like data|
|prod |Product of non-NA values|
|quantile |Compute sample quantile|
|rank |Ordinal ranks of non-NA values, like calling Series.rank|
|size |Compute group sizes, returning result as a Series|
|sum |Sum of non-NA values|
|std, var |Sample standard deviation and variance|

Custom aggregation functions usually are slower than the previous table 
functions.

In [118]:
data

,key1,key2,data1,data2
0,a,1,0.206235,0.221009
1,a,2,-0.820793,-0.267532
2,None,1,-0.374071,-0.546836
3,b,2,-1.691274,1.706690
4,b,1,-0.226503,-1.413782
5,a,<NA>,-0.923427,-0.546473
6,None,1,-0.726968,-0.745141


In [119]:
grouped = data.groupby("key1")

# Extracting the two smollest numbers per key
grouped["data1"].nsmallest(2)

key1   
a     5   -0.923427
      1   -0.820793
b     3   -1.691274
      4   -0.226503
Name: data1, dtype: float64

***
We can use our own aggregation function passing a function which aggregates an 
array to the agg method
***

In [120]:
def range(arr):
    return arr.max() - arr.min()

def trimean(series):
    Q1 = series.quantile(0.25)
    median = series.median()
    Q3 = series.quantile(0.75)
    return (Q1 + 2 * median + Q3) / 4

grouped.agg(trimean)

,key2,data1,data2
key1,,,
a,1.5,-0.705243,-0.241332
b,1.5,-0.958888,0.146454


In [121]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0 -0.512661  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0 -0.958888  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a    -0.307279  0.206235   3.0 -0.197665  0.388482 -0.546473 -0.407003   
b    -0.592696 -0.226503   2.0  0.146454  2.206508 -1.413782 -0.633664   

                                    
           50%       75%       max  
key1                                
a    -0.267532 -0.023261  0.221009  
b     0.146454  0.926572  1.706690  

[2 rows x 24 columns]

### Column-Wise and Multiple Function Application

A DataFrame will have hierarchical columns only f multiple functions are 
applied to at least one column.

In [122]:
with open("datasets/tips.csv") as file:
    data = pd.read_csv(file)

data.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [123]:
# Adding percentage tip column

data["tip_pct"] = data["tip"] / data["total_bill"]
data.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


***
Applying multiple functions to grouped data
***

In [124]:
"""
    mean per day and smoker
"""
grouped = data.groupby(["day", "smoker"])

grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [125]:
grouped_pct.agg(["mean", trimean, "median", "std", range])

mean   trimean    median       std     range
day  smoker                                                  
Fri  No      0.151650  0.149843  0.149241  0.028123  0.067349
     Yes     0.174783  0.172701  0.173913  0.051293  0.159925
Sat  No      0.158048  0.155115  0.150152  0.039767  0.235193
     Yes     0.147906  0.147387  0.153624  0.061375  0.290095
Sun  No      0.160113  0.162074  0.161665  0.042347  0.193226
     Yes     0.187250  0.147323  0.138122  0.154134  0.644685
Thur No      0.160298  0.157392  0.153492  0.038774  0.193350
     Yes     0.163863  0.162642  0.153846  0.039389  0.151240

In [126]:
"""
    Changing names of .agg()
"""

grouped_pct.agg([("average", "mean"), ("trimean", trimean),
                 ("stdev", "std"), ("range", range)])

average   trimean     stdev     range
day  smoker                                        
Fri  No      0.151650  0.149843  0.028123  0.067349
     Yes     0.174783  0.172701  0.051293  0.159925
Sat  No      0.158048  0.155115  0.039767  0.235193
     Yes     0.147906  0.147387  0.061375  0.290095
Sun  No      0.160113  0.162074  0.042347  0.193226
     Yes     0.187250  0.147323  0.154134  0.644685
Thur No      0.160298  0.157392  0.038774  0.193350
     Yes     0.163863  0.162642  0.039389  0.151240

***
By previously establishing a list of functions.

We can pass a list of tuples as well ("name", "function")
***

In [127]:
func = ["count", "mean", "max"]

result = grouped[["tip_pct", "total_bill"]].agg(func)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

***
Also we can apply specific functions to specific collumns passing a dict:
***

In [128]:
grouped.agg({"tip_pct" : ["min", "max", trimean, "std"], 
             "size" : "sum"})

tip_pct                               size
                  min       max   trimean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.149843  0.028123    9
     Yes     0.103555  0.263480  0.172701  0.051293   31
Sat  No      0.056797  0.291990  0.155115  0.039767  115
     Yes     0.035638  0.325733  0.147387  0.061375  104
Sun  No      0.059447  0.252672  0.162074  0.042347  167
     Yes     0.065660  0.710345  0.147323  0.154134   49
Thur No      0.072961  0.266312  0.157392  0.038774  112
     Yes     0.090014  0.241255  0.162642  0.039389   40

### Returning Aggregated Data Without Row Indexes

Not always we want group and that keys be used as index. We can change this
behaviour passing `as_index=False` argument. Also, it is possible to 
obtain this result b calling `reset_index` on the result, but if in the first
place we use `as_index` argument, we avoid some unnecessary computations.

In [129]:
temp = data.groupby(["day", "smoker"], as_index=False)
temp["total_bill"].mean()

,day,smoker,total_bill
0,Fri,No,18.420000
1,Fri,Yes,16.813333
2,Sat,No,19.661778
3,Sat,Yes,21.276667
4,Sun,No,20.506667
5,Sun,Yes,24.120000
6,Thur,No,17.113111
7,Thur,Yes,19.190588


In [130]:
temp = data.groupby(["day", "smoker"])
temp["total_bill"].mean()

day   smoker
Fri   No        18.420000
      Yes       16.813333
Sat   No        19.661778
      Yes       21.276667
Sun   No        20.506667
      Yes       24.120000
Thur  No        17.113111
      Yes       19.190588
Name: total_bill, dtype: float64

In [131]:
temp["total_bill"].mean().reset_index()

,day,smoker,total_bill
0,Fri,No,18.420000
1,Fri,Yes,16.813333
2,Sat,No,19.661778
3,Sat,Yes,21.276667
4,Sun,No,20.506667
5,Sun,Yes,24.120000
6,Thur,No,17.113111
7,Thur,Yes,19.190588


## Apply: General split-apply-combine

The method `apply` is one of the most widely used general purpose methods 
in GroupBy. How works apply: splits the object being manipulated into pieces, 
invokes the passed function on each piece, and then attempts to concatenate 
the pieces.

In [132]:
data.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [133]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

top(data, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [134]:
data.groupby(["smoker", "time"]).apply(top, n=3)

total_bill   tip smoker   day    time  size   tip_pct
smoker time                                                             
No     Dinner 232       11.61  3.39     No   Sat  Dinner     2  0.291990
              51        10.29  2.60     No   Sun  Dinner     2  0.252672
              185       20.69  5.00     No   Sun  Dinner     5  0.241663
       Lunch  149        7.51  2.00     No  Thur   Lunch     2  0.266312
              88        24.71  5.85     No  Thur   Lunch     2  0.236746
              87        18.28  4.00     No  Thur   Lunch     2  0.218818
Yes    Dinner 172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
              178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
              67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       Lunch  221       13.42  3.48    Yes   Fri   Lunch     2  0.259314
              194       16.58  4.00    Yes  Thur   Lunch     2  0.241255
              222        8.58  1.92    Yes   Fri   Lunch     1  0.223776

In [135]:
"""
    Suppressing the Group Keys with 'group_keys=False' argument
"""

data.groupby(["smoker", "time"], group_keys=False).apply(top, n=3)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
149,7.51,2.00,No,Thur,Lunch,2,0.266312
88,24.71,5.85,No,Thur,Lunch,2,0.236746
87,18.28,4.00,No,Thur,Lunch,2,0.218818
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
221,13.42,3.48,Yes,Fri,Lunch,2,0.259314


### Quantile and Bucket Analysis

We can combine `pandas.cut` or `pandas.qcut` with `groupby` to perform bucket
or quantile analysis on a dataset.

In [136]:
data = pd.DataFrame({"data1": np.random.standard_normal(1000),
                     "data2": np.random.standard_normal(1000)})
data.head()

,data1,data2
0,0.871184,-0.314612
1,0.256467,0.463116
2,1.307486,-0.092995
3,0.367515,-2.030566
4,-0.021444,-0.664423


In [137]:
quartiles = pd.cut(data["data1"], 4)

quartiles.head(6)

0    (-0.4, 1.32]
1    (-0.4, 1.32]
2    (-0.4, 1.32]
3    (-0.4, 1.32]
4    (-0.4, 1.32]
5    (-0.4, 1.32]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.848, -2.121] < (-2.121, -0.4] < (-0.4, 1.32] < (1.32, 3.041]]

In [138]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max":group.max(),
         "count": group.count(), "mean":group.mean()}
    )


In [139]:
grouped = data.groupby(quartiles)

grouped.apply(get_stats)

min       max  count      mean
data1                                                      
(-3.848, -2.121] data1 -3.841387 -2.172316     16 -2.577096
                 data2 -2.389906  1.728848     16 -0.111726
(-2.121, -0.4]   data1 -2.064293 -0.402660    311 -0.956233
                 data2 -2.694371  3.505822    311 -0.047400
(-0.4, 1.32]     data1 -0.397904  1.314511    563  0.361686
                 data2 -3.058740  2.785718    563 -0.056667
(1.32, 3.041]    data1  1.323008  3.040684    110  1.773387
                 data2 -2.809103  2.931527    110  0.086509

In [140]:
grouped.agg(["min", "max", "count", "mean"])

data1                               data2            \
                       min       max count      mean       min       max   
data1                                                                      
(-3.848, -2.121] -3.841387 -2.172316    16 -2.577096 -2.389906  1.728848   
(-2.121, -0.4]   -2.064293 -0.402660   311 -0.956233 -2.694371  3.505822   
(-0.4, 1.32]     -0.397904  1.314511   563  0.361686 -3.058740  2.785718   
(1.32, 3.041]     1.323008  3.040684   110  1.773387 -2.809103  2.931527   

                                  
                 count      mean  
data1                             
(-3.848, -2.121]    16 -0.111726  
(-2.121, -0.4]     311 -0.047400  
(-0.4, 1.32]       563 -0.056667  
(1.32, 3.041]      110  0.086509

***
Generating equal bins with 'qcut' and with it's corresponding labels.

Also without labels (`labels=False`)
***

In [141]:
qu_samp = pd.qcut(data["data1"], 4,
                  labels=[f"Q{i+1}" for i in np.arange(0,4)])

qu_samp.head()


0    Q4
1    Q3
2    Q4
3    Q3
4    Q2
Name: data1, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [142]:
grouped = data.groupby(qu_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
Q1    data1 -3.841387 -0.588980    250 -1.200865
      data2 -2.694371  3.505822    250 -0.068181
Q2    data1 -0.583763  0.055983    250 -0.269996
      data2 -2.259076  2.609731    250 -0.027358
Q3    data1  0.055992  0.733638    250  0.369760
      data2 -3.058740  2.782393    250 -0.056136
Q4    data1  0.736263  3.040684    250  1.341420
      data2 -2.809103  2.931527    250 -0.003991

In [143]:
# Now, qcut without labels

qu_samp = pd.qcut(data["data1"], 4,
                  labels=False)

grouped = data.groupby(qu_samp)
group_stats = grouped.apply(get_stats)

# Setting index name to identified them
group_stats.index.names = ["Quartile", "data_col"]
group_stats


min       max  count      mean
Quartile data_col                                     
0        data1    -3.841387 -0.588980    250 -1.200865
         data2    -2.694371  3.505822    250 -0.068181
1        data1    -0.583763  0.055983    250 -0.269996
         data2    -2.259076  2.609731    250 -0.027358
2        data1     0.055992  0.733638    250  0.369760
         data2    -3.058740  2.782393    250 -0.056136
3        data1     0.736263  3.040684    250  1.341420
         data2    -2.809103  2.931527    250 -0.003991

### Example: Filling Missing Values with Group-Specific Values

In [144]:
ser = pd.Series(np.random.standard_normal(6))
ser[::2] = np.nan
"""
    filling NA values with Series mean
"""
print(ser)
ser.fillna(ser.mean())

0         NaN
1   -0.270302
2         NaN
3    1.929969
4         NaN
5    0.722619
dtype: float64


0    0.794095
1   -0.270302
2    0.794095
3    1.929969
4    0.794095
5    0.722619
dtype: float64

In [145]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]

data = pd.Series(np.random.standard_normal(8), index=states)
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio         -0.361123
New York      0.369719
Vermont            NaN
Florida      -1.036101
Oregon       -0.038514
Nevada             NaN
California   -1.051730
Idaho              NaN
dtype: float64

In [146]:
print("Group size:\n", data.groupby(group_key).size(), sep='')
print("\nGroup count:\n", data.groupby(group_key).count(), sep='')
print("\nGroup mean:\n", data.groupby(group_key).mean(), sep='')

Group size:
East    4
West    4
dtype: int64

Group count:
East    3
West    2
dtype: int64

Group mean:
East   -0.342501
West   -0.545122
dtype: float64


In [147]:
def fill_mean(group):
    """
        Fill nan with mean group
    """
    return group.fillna(group.mean())

fill_values = {"East":0.5, "West":-1}
def fill_func(group):
    """
        Fill nan with stablished values
    """
    return group.fillna(fill_values[group.name])

data.groupby(group_key).apply(fill_mean)


East  Ohio         -0.361123
      New York      0.369719
      Vermont      -0.342501
      Florida      -1.036101
West  Oregon       -0.038514
      Nevada       -0.545122
      California   -1.051730
      Idaho        -0.545122
dtype: float64

In [148]:
data.groupby(group_key).apply(fill_func)


East  Ohio         -0.361123
      New York      0.369719
      Vermont       0.500000
      Florida      -1.036101
West  Oregon       -0.038514
      Nevada       -1.000000
      California   -1.051730
      Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

Monte Carlo simulation select a random sample from a dataset.
Here we are going to select random rows from a deck of cards.

In [149]:
suits = ["H", "S", "C", "D"]  # Hearts, Spades, Clubs, Diamonds
card_val = (list(np.arange(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(np.arange(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [150]:
def draw(deck, n=5):
    return deck.sample(n)

def get_suit(card):
    # suit = last letter
    return card[-1]

deck.groupby(get_suit).apply(draw, n=2)

C  10C    10
   QC     10
D  QD     10
   6D      6
H  3H      3
   5H      5
S  9S      9
   8S      8
dtype: int64

In [151]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

2C     2
QC    10
2D     2
6D     6
2H     2
KH    10
8S     8
QS    10
dtype: int64

### Example: Group-Wise Linear Regression

In [152]:
def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

data = pd.read_pickle("datasets/eg_yahoo_price.pkl")

data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1714 entries, 2010-01-04 to 2016-10-21
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    1714 non-null   float64
 1   GOOG    1714 non-null   float64
 2   IBM     1714 non-null   float64
 3   MSFT    1714 non-null   float64
dtypes: float64(4)
memory usage: 67.0 KB


In [153]:
grouped = data.groupby(data.index.year)

grouped.apply(regress, yvar="AAPL", xvars=["IBM"])

,IBM,intercept
Date,,
2010,0.639765,-38.934577
2011,0.255283,9.218575
2012,0.942581,-90.172620
2013,-0.495341,150.731177
2014,-0.670520,202.155796
2015,0.521980,39.535025
2016,0.318604,55.345271


## Group Transforms and "Unwrapped" GroupBys

The method `transform()` works like `apply()` but with more constraints:
- Produce an object of the same shape as the input group.
- It must not mutate its input.
- It can produce a scalar value to be broadcast to the shape of the group.

We can benefit from aggregate functions using `transform()` compared to `apply()`, these functions will be faster (sum, mean...). This allows us to perform what is called an *unwrapped* group operations.

In [154]:
data = pd.DataFrame({"key": ['a', 'b', 'c'] * 4,
                     "value": np.arange(12.)})

data.sample(5)

,key,value
11,c,11.0
9,a,9.0
3,a,3.0
6,a,6.0
1,b,1.0


In [155]:
grouped = data.groupby("key")["value"]
grouped.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [156]:
grouped.agg("mean")

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [157]:
grouped.transform("mean")

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [158]:
def double(group):
    return group * 2

grouped.transform(double)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [159]:
#grouped.apply(rank, ascending=False)
grouped.rank(ascending=False)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

## Pivot Tables and Cross-Tabulation

DataFrame has a `pivot_table()` method. Pivot Table aggregates a table of data 
by one or more keys, arranging the data in a rectangle with some of the group 
keys along the rows and some along the columns. Also there are a top-level
function `pandas.pivot_table`. We can see it like an interface for groupby, 
`pivot_table` can add partial totals (*margins*).

*pivot_table options*
|Argument|Description|
|--:|---|
|values |Column name or names to aggregate; by default, aggregates all numeric columns|
|index |Column names or other group keys to group on the rows of the resulting pivot table|
|columns |Column names or other group keys to group on the columns of the resulting pivot table|
|aggfunc |Aggregation function or list of functions ("mean" by default); can be any function valid in a groupby context|
|fill_value |Replace missing values in the result table|
|dropna |If True, do not include columns whose entries are all NA|
|margins |Add row/column subtotals and grand total (False by default)|
|margins_name |Name to use for the margin row/column labels when passing margins=True; defaults to "All"|
|observed |With Categorical group keys, if True, show only the observed category values in the keys rather than all categories|

In [160]:
with open("datasets/tips.csv") as file:
    data = pd.read_csv(file)

data["tip_pct"] = data["tip"] / data["total_bill"]
temp = data[["total_bill", "tip", "smoker", "day", "size"]]
temp.head()



,total_bill,tip,smoker,day,size
0,16.99,1.01,No,Sun,2
1,10.34,1.66,No,Sun,3
2,21.01,3.50,No,Sun,3
3,23.68,3.31,No,Sun,2
4,24.59,3.61,No,Sun,4


In [161]:
day_smoker = ["day", "smoker"]
pd.pivot_table(data=temp, index=day_smoker)

size       tip  total_bill
day  smoker                                
Fri  No      2.250000  2.812500   18.420000
     Yes     2.066667  2.714000   16.813333
Sat  No      2.555556  3.102889   19.661778
     Yes     2.476190  2.875476   21.276667
Sun  No      2.929825  3.167895   20.506667
     Yes     2.578947  3.516842   24.120000
Thur No      2.488889  2.673778   17.113111
     Yes     2.352941  3.030000   19.190588

In [162]:
"""
    This groupby() is similar to previous pivot_table()
"""
temp.groupby(["day", "smoker"]).mean()

total_bill       tip      size
day  smoker                                
Fri  No       18.420000  2.812500  2.250000
     Yes      16.813333  2.714000  2.066667
Sat  No       19.661778  3.102889  2.555556
     Yes      21.276667  2.875476  2.476190
Sun  No       20.506667  3.167895  2.929825
     Yes      24.120000  3.516842  2.578947
Thur No       17.113111  2.673778  2.488889
     Yes      19.190588  3.030000  2.352941

In [163]:
data.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"])



size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

***
We can add partial totals (adds 'All' row and columns) with `margins=True`
argument.
***

In [164]:
data.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"], margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

***
Istead of `mean` we can pass other functions with `aggfunc=` argument.
***

In [165]:
data.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"], aggfunc=len, margins=True)

# cross-tabulation frecuency of group sizes

size            tip_pct           
smoker          No   Yes  All      No   Yes  All
time   day                                      
Dinner Fri     3.0   9.0   12     3.0   9.0   12
       Sat    45.0  42.0   87    45.0  42.0   87
       Sun    57.0  19.0   76    57.0  19.0   76
       Thur    1.0   NaN    1     1.0   NaN    1
Lunch  Fri     1.0   6.0    7     1.0   6.0    7
       Thur   44.0  17.0   61    44.0  17.0   61
All          151.0  93.0  244   151.0  93.0  244

### Cross-Tabulations: Crosstab

To summarize data we can use `pandas.crosstab` function similar to pivot_table
but sometimes more convenient.

In [166]:
tab_data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""

tap_data = pd.read_table(StringIO(data), sep="\s+")
tap_data

TypeError: initial_value must be str or None, not DataFrame

In [ ]:
pd.crosstab(tap_data["Nationality"], tap_data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [167]:
pd.crosstab([data["time"], data["day"]], data["smoker"], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244